In [1]:
import jax.numpy as jnp
import jax.ops
import scipy as scipy
import scipy.integrate
import scipy.optimize
from jax import jit
from functools import partial
import numpy as np
from jax.experimental.ode import odeint
from ticktack import load_presaved_model, Box, Flow, CarbonBoxModel
import numpy as np
import matplotlib.pyplot as plt
from jax.lax import cond, dynamic_update_slice, fori_loop, dynamic_slice

In [2]:
a = jnp.array([1,2,3])
type(a)

jaxlib.xla_extension.DeviceArray

In [ ]:
@partial(jit, static_argnums=(0,2,3, 5, 6,7 ))
def run(cbm, time_out, oversample,production, y0=None, args=(), target_C_14=None, steady_state_production=None):
        @jit
        def derivative(y, t):
            ans = jnp.matmul(cbm._matrix, y)
            production_rate_constant = production(t, *args)
            production_rate_constant = cbm._convert_production_rate(production_rate_constant)
            production_term = cbm._production_coefficients * production_rate_constant
            return ans + production_term

        time_out = jnp.array(time_out)
        time_values = jnp.linspace(jnp.min(time_out) - 1, jnp.max(time_out) + 1, (time_out.shape[0] + 1) * oversample)
        solution = None
        if y0 is not None:
            y_initial = jnp.array(y0)
        else:
            if steady_state_production is not None:
                solution = cbm.equilibrate(production_rate=steady_state_production)

            elif target_C_14 is not None:
                solution = cbm.equilibrate(production_rate=cbm.equilibrate(target_C_14=target_C_14))
            else:
                ValueError("Must give either target C-14 or production rate.")
            y_initial = jnp.array(solution)

        if not callable(production):
            raise ValueError("incorrect object type for production")

        states = odeint(derivative, y_initial, time_values, atol=1e-15, rtol=1e-15)
        return states, solution, time_values